In [23]:
!pip install catboost scikit-learn xgboost imblearn optuna -q

In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from imblearn.combine import SMOTETomek
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Preprocessing
df = pd.read_csv('/home/workspace/-24633600_1327689908.csv')
# df = pd.read_csv('/content/drive/MyDrive/Projects/TurboFan_20241208_1/-24633600_1327689908.csv')

X = df.drop(['Label'], axis=1)
y = df['Label']

random_state = 42
random_seed = 42
seed = 8

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=10)
dbscan_labels = dbscan.fit_predict(X_scaled)

mask = dbscan_labels != -1
X_filtered = X_scaled[mask]
y_filtered = y[mask]

top_features = [7, 4, 1, 0, 6]
X_filtered_selected = X_filtered[:, top_features]

X_train, X_test, y_train, y_test = train_test_split(
    X_filtered_selected, y_filtered, test_size=0.2, random_state= random_state, stratify=y_filtered
)

smote_tomek = SMOTETomek(random_state=random_state)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)

import optuna

def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'depth': trial.suggest_int('depth', 3, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 20),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 0, 1),
        'loss_function': 'MultiClass',
        'verbose': 0,
        'random_seed': random_seed
    }
    model = CatBoostClassifier(**param)
    model.fit(X_resampled, y_resampled)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred, average='macro')

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=seed))
study.optimize(objective, n_trials=70)
best_params = study.best_params
print("Best Parameters from Optuna:", best_params)

cat_model = CatBoostClassifier(**best_params, verbose=0, random_seed=random_seed)
cat_model.fit(X_resampled, y_resampled)
y_pred_cat = cat_model.predict(X_test)
print("F1 Score (macro) for CatBoost:", f1_score(y_test, y_pred_cat, average='macro'))

rf_model = RandomForestClassifier(random_state=random_state, n_estimators=100, max_depth=5)
gb_model = GradientBoostingClassifier(random_state=random_state, n_estimators=100, max_depth=5)

stacking_clf = StackingClassifier(
    estimators=[('cat', cat_model), ('rf', rf_model), ('gb', gb_model)],
    final_estimator=LogisticRegression(),
    cv=5
)
stacking_clf.fit(X_resampled, y_resampled)
y_pred_stack = stacking_clf.predict(X_test)
print("F1 Score (macro) for StackingClassifier:", f1_score(y_test, y_pred_stack, average='macro'))

/usr/local/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.12/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTETomek or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.vali

Best Parameters from Optuna: {'iterations': 1800, 'learning_rate': 0.08901253643210584, 'depth': 6, 'l2_leaf_reg': 8.105501613485234, 'border_count': 137, 'random_strength': 0.23602215621504516}
F1 Score (macro) for CatBoost: 0.8854700854700855
F1 Score (macro) for StackingClassifier: 0.8497435897435898
